<a href="https://colab.research.google.com/github/NSuprotivniy/2017-big-data/blob/master/autoencoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install git+https://github.com/rcmalli/keras-vggface.git

  Cloning https://github.com/rcmalli/keras-vggface.git to /tmp/pip-req-build-xfo8xg7l
  Stored in directory: /tmp/pip-ephem-wheel-cache-yvofgm87/wheels/36/07/46/06c25ce8e9cd396dabe151ea1d8a2bc28dafcb11321c1f3a6d
Successfully built keras-vggface


In [2]:
from keras.engine import  Model
from keras.layers import Flatten, Dense, Input, Dropout, Reshape
from keras_vggface.vggface import VGGFace
from keras.utils import Sequence
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
import tensorflow as tf
from keras import backend as K

Using TensorFlow backend.


In [0]:
import os
from glob import glob
import numpy as np
from google.colab import drive
import math

In [0]:
def get_model():
  vgg_model = VGGFace(include_top=False, input_shape=(224, 224, 3), model='resnet50')
  vgg_model.trainable = False
  last_layer = vgg_model.get_layer('avg_pool').output
  x = Flatten(name='flatten')(last_layer)
  out = Dense(2048, activation='relu')(x)
  encoder = Model(vgg_model.input, out, name="resnet50_faces_encoder")
  
  input_encoded = Input(shape=(2048,))
  flat_decoded = Dense(224*224*3, activation='relu')(input_encoded)
  decoded = Reshape((224, 224, 3))(flat_decoded)
  
  decoder = Model(input_encoded, decoded)

  autoencoder = Model(vgg_model.input, decoder(encoder(vgg_model.input)))
  autoencoder.compile(optimizer=Adam(lr=0.000001), loss='binary_crossentropy')
  return encoder, autoencoder

In [5]:
drive.mount("/content/nsuprotivniy/")
data_path = "/content/nsuprotivniy/My Drive/Colab Notebooks/data/LCC_FASD/"

Drive already mounted at /content/nsuprotivniy/; to attempt to forcibly remount, call drive.mount("/content/nsuprotivniy/", force_remount=True).


In [0]:
class Generator(Sequence):  
  def __init__(self, batch_size = 2):
    files = glob(os.path.join(data_path, "*/*/cropped/*.npy")) + glob(os.path.join(data_path, "LCC_FASD_evaluation/cropped/*.npy"))
    self.batches = np.array_split(files, math.ceil(len(files) / batch_size))
    
  def __len__(self):
    return len(self.batches)

  def __getitem__(self, idx):
    X = np.array([np.load(path) for path in self.batches[idx]])
    return X, X

In [0]:
gen = Generator()

In [0]:
model_path = os.path.join(data_path, "autoencoder")
if (not os.path.exists(model_path)):
  os.mkdir(model_path)

In [0]:
checkpoints_path = os.path.join(model_path, "checkpoints")
if (not os.path.exists(checkpoints_path)):
  os.mkdir(checkpoints_path)

In [0]:
checkpoint_filename = "autoencoder_model_weights.h5"
checkpoint = [ModelCheckpoint(os.path.join(checkpoints_path, checkpoint_filename), verbose=1, period=5)]
encoder, autoencoder = get_model()
autoencoder.fit_generator(gen, epochs=100, verbose=1, callbacks=checkpoint)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/100
9414/9414 [==============================] - 3506s 372ms/step - loss: -1288.1002
Epoch 2/100
 245/9414 [..............................] - ETA: 46:07 - loss: -1884.6891

In [0]:
model_json = encoder.to_json()
with open(os.path.join(model_path, "encoder.json"), "w") as json_file:
    json_file.write(model_json)
    
encoder.save_weights(os.path.join(model_path, "encoder.h5"))